## The Data

In [ ]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 27.4 MB/s eta 0:00:00


In [ ]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
import implicit
import os

/usr/local/lib/python3.10/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [ ]:
os.chdir('/content/drive/MyDrive/DACON_국민대AI/study/mysong/코드 예시/ALS_implicit_code')

In [ ]:
retail_df = pd.read_excel('data/Online Retail.xlsx')
retail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [ ]:
retail_df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [ ]:
retail_df = retail_df[retail_df['CustomerID'].notna()]
grouped_df = retail_df[['CustomerID', 'StockCode', 'Description', 'Quantity']].groupby(['CustomerID', 'StockCode', 'Description']).sum().reset_index()
grouped_df.loc[grouped_df['Quantity'] == 0, ['Quantity']] = 1
grouped_df = grouped_df.loc[grouped_df['Quantity'] > 0]

grouped_df

,CustomerID,StockCode,Description,Quantity
0,12346.0,23166,MEDIUM CERAMIC TOP STORAGE JAR,1
1,12347.0,16008,SMALL FOLDING SCISSOR(POINTED EDGE),24
2,12347.0,17021,NAMASTE SWAGAT INCENSE,36
3,12347.0,20665,RED RETROSPOT PURSE,6
4,12347.0,20719,WOODLAND CHARLOTTE BAG,40
...,...,...,...,...
269265,18287.0,72351B,SET/6 PINK BUTTERFLY T-LIGHTS,24
269266,18287.0,84507C,BLUE CIRCLES DESIGN MONKEY DOLL,6
269267,18287.0,85039A,SET/4 RED MINI ROSE CANDLE IN BOWL,96
269268,18287.0,85039B,S/4 IVORY MINI ROSE CANDLE IN BOWL,120


In [ ]:
import plotly.express as px

fig = px.histogram(grouped_df, x='Quantity', title='Distribution of the purchase quantity', nbins=500)
fig.show();

In [ ]:
print(f'Number of unique customers: {grouped_df.CustomerID.nunique()}')
print(f'Number of unique items: {grouped_df.StockCode.nunique()}')

print(f'Average purchase quantity per interaction: {int(grouped_df.Quantity.mean())}')
print(f'Minimum purchase quantity per interaction: {grouped_df.Quantity.min()}')
print(f'Maximum purchase quantity per interaction: {grouped_df.Quantity.max()}')

Number of unique customers: 4338
Number of unique items: 3664
Average purchase quantity per interaction: 18
Minimum purchase quantity per interaction: 1
Maximum purchase quantity per interaction: 12540


## Implicit Feedback

In [ ]:
unique_customers = grouped_df.CustomerID.unique()
customer_ids = dict(zip(unique_customers, np.arange(unique_customers.shape[0], dtype=np.int32)))

unique_items = grouped_df.StockCode.unique()
item_ids = dict(zip(unique_items, np.arange(unique_items.shape[0], dtype=np.int32)))

grouped_df['customer_id'] = grouped_df.CustomerID.apply(lambda i: customer_ids[i])
grouped_df['item_id'] = grouped_df.StockCode.apply(lambda i: item_ids[i])

sparse_item_customer = sparse.csr_matrix((grouped_df['Quantity'].astype(float), (grouped_df['item_id'], grouped_df['customer_id'])))
sparse_customer_item = sparse.csr_matrix((grouped_df['Quantity'].astype(float), (grouped_df['customer_id'], grouped_df['item_id'])))

model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)

alpha = 15
data = (sparse_item_customer * alpha).astype('double')

model.fit(data)

/usr/local/lib/python3.10/dist-packages/implicit/cpu/als.py:95: RuntimeWarning:

OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.



  0%|          | 0/50 [00:00<?, ?it/s]

## Example of Recommendation - Finding the Similar Items

In [ ]:
grouped_df.loc[grouped_df['item_id'] == 1319].head()

,CustomerID,StockCode,Description,Quantity,customer_id,item_id
3078,12409.0,71053,WHITE METAL LANTERN,12,50,1319
6411,12462.0,71053,WHITE METAL LANTERN,4,95,1319
11523,12556.0,71053,WHITE METAL LANTERN,2,166,1319
22679,12748.0,71053,WHITE METAL LANTERN,1,326,1319
25856,12840.0,71053,WHITE METAL LANTERN,4,390,1319


In [ ]:
item_id = 1319
n_similar = 10

item_vecs = model.item_factors
customer_vecs = model.user_factors

item_norms = np.sqrt((item_vecs * item_vecs).sum(axis=1))

scores = item_vecs.dot(item_vecs[item_id]) / item_norms
top_idx = np.argpartition(scores, -n_similar)[-n_similar:]
similar = sorted(zip(top_idx, scores[top_idx] / item_norms[item_id]), key=lambda x: -x[1])

for item in similar:
    idx, score = item
    print(grouped_df.Description.loc[grouped_df.item_id == idx].iloc[0])

WHITE METAL LANTERN
FLAG OF ST GEORGE CHAIR
CARD BIRTHDAY COWBOY
ST GEORGE SET OF 10 PARTY LIGHTS
SET 4 PICNIC CUTLERY BLUEBERRY
SET/4 SKULL BADGES
DOORMAT 3 SMILEY CATS
PLASTERS IN TIN WOODLAND ANIMALS
OCEAN STRIPE HAMMOCK 
PINK BUTTERFLY CUSHION COVER 


## Example of Recommendation - Recommend Items to Customers

In [ ]:
grouped_df.loc[grouped_df['customer_id'] == 2].sort_values('Quantity', ascending=False)[['customer_id', 'Description', 'Quantity']].head(20)

,customer_id,Description,Quantity
120,2,DOUGHNUT LIP GLOSS,200
121,2,ICE CREAM PEN LIP GLOSS,192
114,2,PACK OF 12 HEARTS DESIGN TISSUES,144
112,2,PACK OF 12 SUKI TISSUES,144
118,2,60 CAKE CASES VINTAGE CHRISTMAS,144
105,2,PACK OF 72 SKULL CAKE CASES,144
113,2,PACK OF 12 BLUE PAISLEY TISSUES,144
123,2,60 TEATIME FAIRY CAKE CASES,144
111,2,PACK OF 12 WOODLAND TISSUES,144
110,2,PACK OF 12 RED RETROSPOT TISSUES,144
